In [27]:
#imports
import pandas as pd
from utils import list_folders, json_to_dataframe, list_all_files

main_folder = 'snapshots'
dates = list_folders(main_folder)
files = list_all_files(main_folder, dates)

#stations_data = json_to_polar_dataframe(files)
stations_data = json_to_dataframe(files)
# Extact the columns timestamp, station_id, num_bikes_available


In [28]:
# ALARM 1: BIKES DISABLED
# Description: An alarm 
# bikes_disabled > 5
bikes_limit = 5
bikes_disabled = stations_data[stations_data['num_bikes_disabled'] > bikes_limit]
bikes_disabled

,is_renting,is_installed,num_docks_available,traffic,last_reported,is_returning,num_docks_disabled,is_charging_station,status,station_id,num_bikes_available,num_bikes_disabled,num_bikes_available_types.ebike,num_bikes_available_types.mechanical,file,timestamp_file
24,1,1,10,None,1694063271,1,0,True,IN_SERVICE,26,2,6,0,2,snapshots/2023-09-07/1694063435.json,1694063435
40,1,1,11,None,1694063209,1,0,True,IN_SERVICE,43,8,9,3,5,snapshots/2023-09-07/1694063435.json,1694063435
78,1,1,7,None,1694063321,1,0,True,IN_SERVICE,82,25,8,2,23,snapshots/2023-09-07/1694063435.json,1694063435
123,1,1,16,None,1694063211,1,0,True,IN_SERVICE,128,21,6,1,20,snapshots/2023-09-07/1694063435.json,1694063435
139,1,1,10,None,1694063416,1,0,True,IN_SERVICE,144,11,6,4,7,snapshots/2023-09-07/1694063435.json,1694063435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,1,1,22,None,1693689500,1,0,True,IN_SERVICE,353,3,6,3,0,snapshots/2023-09-02/1693689654.json,1693689654
354,1,1,10,None,1693689588,1,0,True,IN_SERVICE,362,16,6,1,15,snapshots/2023-09-02/1693689654.json,1693689654
363,1,1,25,None,1693689513,1,0,True,IN_SERVICE,371,3,8,0,3,snapshots/2023-09-02/1693689654.json,1693689654
440,1,1,19,None,1693689536,1,0,True,IN_SERVICE,449,2,6,0,2,snapshots/2023-09-02/1693689654.json,1693689654


In [30]:
# status IN ('IN_SERVICE', 'NOT_IN_SERVICE', 'MAINTEINANCE')
# ALARM 2: STATION OUT OF SERVICE
# Description: An alarm is triggered when a station goes out of service
def get_outages(df):
    outages = []
    for station_id in df['station_id'].unique():
        station_data = df[df['station_id'] == station_id]
        starts = station_data[station_data['start_outage']]
        ends = station_data[station_data['end_outage']]
        for start in starts.itertuples():  
            filtered_ends = ends[ends['timestamp'] > start.timestamp]
            if not filtered_ends.empty:
                end = filtered_ends.iloc[0]  
                outages.append((station_id, start.timestamp, end.timestamp))
            else:
                outages.append((station_id, start.timestamp, None))
    return pd.DataFrame(outages, columns=['station_id', 'begin_timestamp', 'end_timestamp'])


outages = stations_data
outages['timestamp_file'] = pd.to_datetime(outages['timestamp_file'], unit='s')
outages.sort_values(by=['station_id', 'timestamp_file'], inplace=True)
outages['status_shifted'] = outages.groupby('station_id')['status'].shift(-1)
outages['start_outage'] = (outages['status'] == 'IN_SERVICE') & (outages['status_shifted'] != 'IN_SERVICE')
outages['end_outage'] = (outages['status'] != 'IN_SERVICE') & (outages['status_shifted'] == 'IN_SERVICE')
outages.rename(columns={'timestamp_file': 'timestamp'}, inplace=True)
outages = outages[['station_id', 'status','status_shifted', 'timestamp', 'start_outage', 'end_outage']]
outages = outages[(outages['start_outage'] == True) | (outages['end_outage'] == True)] 
outages = outages[outages['status_shifted'].notnull()]
outages = get_outages(outages)
outages['diff'] = outages['end_timestamp'] - outages['begin_timestamp']
outages.head(5)


KeyError: 'timestamp_file'